# Assignment

# Naive Bayes Classification

## 1. 직접 수식을 작성하여 Naive Bayes 모델을 만들겠습니다.

- 입력문서가 {fast, fly, shoot} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 장르로 분류가 될지 알아보겠습니다.
- 주어진 dataset 을 확인해 보기위해 엑셀 파일을 읽습니다.

In [60]:
import numpy as np
import pandas as pd

In [61]:
documents = pd.read_excel('./test_file.xlsx') 
documents

,label,message
0,comedy,"Fun couple, love love"
1,action,"fast Furious, shoot!!"
2,comedy,"couple^^, fly, fast, fun, fun"
3,action,"furious, shoot shoot fun"
4,action,"fly~~ fast shoot, Love"


### step 1. 단어를 모두 소문자로 바꾸어 줍니다. 
* message 칼럼에 있는 단어들을 모두 소문자로 바꿔주세요.

In [62]:
lower_documents = [string.lower() for string in documents["message"]]

print(lower_documents)

['fun couple, love love', 'fast furious, shoot!!', 'couple^^, fly, fast, fun, fun', 'furious, shoot shoot fun', 'fly~~ fast shoot, love']


### step2. string.punctuation 을 이용해서 특수문자를 제거해주세요.
- string.punctuation 에는 특수문자들이 저장되어있는것을 확인할수 있습니다.

In [63]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [64]:
import re

del_punctuation_documents = []

for words in lower_documents:
    new_string = ''.join([i for i in words if i not in string.punctuation]) ##특수문자가 붙어있지 않은 단어만을 선별해 리스트에 붙임.
    del_punctuation_documents.append(new_string)

print(del_punctuation_documents)

['fun couple love love', 'fast furious shoot', 'couple fly fast fun fun', 'furious shoot shoot fun', 'fly fast shoot love']


### step 3. 단어를 하나씩 띄어쓰기 단위로 쪼개어 줍니다.

In [65]:
preprocessed_documents=[d.split(' ') for d in del_punctuation_documents] #위의 특수문자를 제거한 리스트열의 단어를 다시 돌아가며 뛰어쓰기 단위로 split

print(preprocessed_documents)

[['fun', 'couple', 'love', 'love'], ['fast', 'furious', 'shoot'], ['couple', 'fly', 'fast', 'fun', 'fun'], ['furious', 'shoot', 'shoot', 'fun'], ['fly', 'fast', 'shoot', 'love']]


### step 4. 문자를 숫자로 변환해줍니다.
- 이를 위해 CountVectorizer를 사용합니다.
- CountVectorizer는 문서 집합에서 단어 토큰을 생성하고, 각 단어 수를 세어 BOW (Bag of Words) 인코딩한 벡터를 만들어줍니다.

- 문서에 해당단어가 몇번 포함되었는지를 나타낼 때 사용하는 방법입니다.

- 참고 : https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

![CountVectorized](https://user-images.githubusercontent.com/68625698/106378540-15d8ed80-63e9-11eb-8604-5c960c274867.PNG)

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(documents['message'])

CountVectorizer()

In [67]:
doc_array = count_vector.transform(documents['message']).toarray() #label분류할 대상인 message를 숫자로 변환
doc_array

array([[1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 2],
       [0, 1, 1, 0, 0, 1, 1]])

In [68]:
count_vector.get_feature_names() #무슨 단어를 기준으로 열마다 나누어져있는지 확인

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot']

In [69]:
# doc_array 배열에 순서에 따라 매치되는 단어를 나타낸다.
count_vector.vocabulary_

{'couple': 0,
 'fast': 1,
 'fly': 2,
 'fun': 3,
 'furious': 4,
 'love': 5,
 'shoot': 6}

In [70]:
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,couple,fast,fly,fun,furious,love,shoot
0,1,0,0,1,0,2,0
1,0,1,0,0,1,0,1
2,1,1,1,2,0,0,0
3,0,0,0,1,1,0,2
4,0,1,1,0,0,1,1


In [71]:
frequency_matrix['count'] = frequency_matrix.sum(axis=1) ##한 문장에 단어가 몇개쓰였는지 보여주는 count변수 생성
frequency_matrix

,couple,fast,fly,fun,furious,love,shoot,count
0,1,0,0,1,0,2,0,4
1,0,1,0,0,1,0,1,3
2,1,1,1,2,0,0,0,5
3,0,0,0,1,1,0,2,4
4,0,1,1,0,0,1,1,4


### step 5. 범주형 변수를 dummy변수로 변환해주는 작업(One-Hot Encoding!)을 해주어야합니다.
- label 을 comedy =1, action =0 으로 변환해주세요

In [72]:
documents['label'] = documents['label'].map({'comedy': 1, 'action': 0})

In [73]:
# documents['label'],frequency_matrix 결합
doc = pd.concat([documents['label'],frequency_matrix],axis=1) 

In [74]:
doc

,label,couple,fast,fly,fun,furious,love,shoot,count
0,1,1,0,0,1,0,2,0,4
1,0,0,1,0,0,1,0,1,3
2,1,1,1,1,2,0,0,0,5
3,0,0,0,0,1,1,0,2,4
4,0,0,1,1,0,0,1,1,4


###  step 6. 나이브 베이즈 계산을 해봅시다!

- 입력문서가 {fast, fly, shoot} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 장르로 분류가 될까요? (계산과정을 채워주세요) 


In [87]:
doc = np.array(doc)      # dataframe을 np-array로 변환해줍니다.
                         # return값은 np-matrix가 아니라 np-array입니다.
doc

/bin/bash: -c: line 0: syntax error near unexpected token `attachment:image.png'
/bin/bash: -c: line 0: `[image.png](attachment:image.png)'


In [76]:
type(doc)

numpy.ndarray

####  P(Y=comedy), P(Y=action) 계산하기

In [77]:
# P(Y=comedy) comedy=1
p_comedy = sum(doc[:,0]==1) / len(doc) 

# P(Y=action)action은 0
p_action = sum(doc[:,0]==0) / len(doc)

In [78]:

print('p_comedy : ',p_comedy)
print('p_action : ', p_action)

p_comedy :  0.4
p_action :  0.6


#### P(fast=1|comedy=1), P(fly=1|comedy=1), P(shoot=1|comedy=1) 계산하기

* 문서에 두번 등장한 단어 주의
* 조건부 확률 계산식 참고 


In [79]:
#1열은 lable확인,각각2,3,7열 fast,fly,shoot열
# P(fast=1|comedy=1)
p_comedy_fast = (sum(doc[(doc[:, 0] == 1) & (doc[:, 2] >= 1)][:, 2])) / sum(doc[doc[:, 0] == 1][:, -1])

# P(fly=1|comedy=1)
p_comedy_fly = (sum(doc[(doc[:, 0] == 1) & (doc[:, 3] >= 1)][:, 3])) / sum(doc[doc[:, 0] == 1][:, -1])

# P(shoot=1|comedy=1)
p_comedy_shoot = (sum(doc[(doc[:, 0] == 1) & (doc[:, 7] >= 1)][:, 7])) / sum(doc[doc[:, 0] == 1][:, -1])

In [80]:
print('p_comedy_fast : ' , p_comedy_fast)
print('p_comedy_fly : ' , p_comedy_fly)
print('p_comedy_shoot : ' , p_comedy_shoot)   

p_comedy_fast :  0.1111111111111111
p_comedy_fly :  0.1111111111111111
p_comedy_shoot :  0.0


#### P(fast=1|action=1), P(fly=1|action=1), P(shoot=1|action=1) 계산하기

In [81]:
# P(fast=1|action=1)
p_action_fast = (sum(doc[(doc[:, 0] == 0) & (doc[:, 2] >= 1)][:, 2])) / sum(doc[doc[:, 0] == 0][:, -1])
# P(fly=1|action=1)
p_action_fly = (sum(doc[(doc[:, 0] == 0) & (doc[:, 3] >= 1)][:, 3])) / sum(doc[doc[:, 0] == 0][:, -1])
# P(shoot=1|action=1)
p_action_shoot = (sum(doc[(doc[:, 0] == 0) & (doc[:, 7] >= 1)][:, 7])) / sum(doc[doc[:, 0] == 0][:, -1])

In [82]:
print('p_action_fast : ' , p_action_fast)
print('p_action_fly : ' , p_action_fly)
print('p_action_shoot : ' , p_action_shoot)

p_action_fast :  0.18181818181818182
p_action_fly :  0.09090909090909091
p_action_shoot :  0.36363636363636365


#### P(Y = comedy| X = fast, fly, shoot) , P(Y = action=1| X = fast, fly, shoot) 값 계산하기

In [83]:
#P(Y = comedy| X = fast, fly, shoot)
proba_comedy = p_comedy_fast*p_comedy_fly*p_comedy_shoot*p_comedy

#P(Y = action=1| X = fast, fly, shoot)
proba_action = p_action_fast*p_action_fly*p_action_shoot*p_action

In [84]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy 0.0
proba_action 0.0036063110443275734


### step 7. 라플라스 스무딩
1. P(Y = comedy| X = fast, fly, shoot) => 0
2. P(Y = action| X = fast, fly, shoot) => 0.003606311044327574

- proba_comedy를 보면 p_comedy_shoot의 값인 0으로 인해 확률이 0으로 계산되었다는 것을 확인할 수 있습니다. 이 문제점을 해결해주세요.

라플라스 스무딩이란?
앞과 같은 상황(likelihood=0)을 막기 위해 최소한의 확률을 설정해줌.


In [86]:
#분모에는 입력변수들의 개수7,분자에는 +1처리
# P(fast=1|comedy=1)
p_comedy_fast_L =  (sum(doc[(doc[:, 0] == 1) & (doc[:, 2] >= 1)][:, 2]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + 7)

# P(fly=1|comedy=1)
p_comedy_fly_L =(sum(doc[(doc[:, 0] == 1) & (doc[:, 3] >= 1)][:, 3]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + 7)

# P(shoot=1|comedy=1)
p_comedy_shoot_L = (sum(doc[(doc[:, 0] == 1) & (doc[:, 7] >= 1)][:, 7]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + 7)

In [ ]:
print('p_comedy_fast_L : ' , p_comedy_fast_L)
print('p_comedy_fly_L : ' , p_comedy_fly_L)
print('p_comedy_shoot_L : ' , p_comedy_shoot_L)

In [ ]:
# P(fast=1|action=1)
p_action_fast_L = (sum(doc[(doc[:, 0] == 0) & (doc[:, 2] >= 1)][:, 2]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + 7)

# P(fly=1|action=1)
p_action_fly_L = (sum(doc[(doc[:, 0] == 0) & (doc[:, 3] >= 1)][:, 3]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + 7)
# P(shoot=1|action=1)
p_action_shoot_L =(sum(doc[(doc[:, 0] == 0) & (doc[:, 7] >= 1)][:, 7]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + 7)

In [ ]:
print('p_action_fast_L : ' , p_action_fast_L)
print('p_action_fly_L : ' , p_action_fly_L)
print('p_action_shoot_L : ' , p_action_shoot_L)

In [ ]:
#P(Y = comedy| X = fast, fly, shoot)
proba_comedy_L =  p_comedy_fast_L*p_comedy_fly_L*p_comedy_shoot_L*p_comedy

#P(Y = action=1| X = fast, fly, shoot)
proba_action_L = p_action_fast_L*p_action_fly_L*p_action_shoot_L*p_action


In [ ]:
print('proba_comedy_L : ', proba_comedy_L)
print('proba_action_L : ', proba_action_L)

### => 두개의 값을 비교하면 action 장르일 때 fast, fly, shoot 단어가 더 자주 나온다고 할 수 있다.

## 2. sklearn을 활용한 Naive Bayes Model
- 먼저 수업 시간에 활용했던 날씨와 온도 데이터를 활용해보겠습니다.

In [ ]:
weather=['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny',
'Rainy','Sunny','Overcast','Overcast','Rainy']
temp=['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']

play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

In [ ]:
df = pd.DataFrame({'weather':weather, 'temperature':temp, 'play':play})
df #DataFrame화

### step 1. 문자를 Label Encoding 해줍니다.

-label encoder
문자를 0부터 시작하는 정수형 숫자로 바꿔줌.
반대로,라벨의 코드숫자를 이용해 원본값을 구할 수도 있음.

In [ ]:
from sklearn.preprocessing import LabelEncoder


"""
라벨 인코더를 생성, fitting하여 
weather, temp, play 에 대해 라벨 인코딩 해주세요.
"""
le = LabelEncoder() #label encoder 생성
encoded=[] #encode된 컴럼을 넣을 빈 리스트 생성
cols = ['weather', 'temperature', 'play']

for i in cols: #세가지의 컬럼을 돌아가며
  le.fit(df[i].unique()) #fit하고
  encoded.append(le.transform(df[i]))  #라벨숫자로 변환하여 빈 리스트에 추가
#
# Encode labels of multiple columns at once
#
df.head()

weather_encoded = encoded[0]
temp_encoded = encoded[1]
label = encoded[2]

print("weather:", weather_encoded)
print("Temp:",temp_encoded)
print("Play:",label)

In [ ]:
features = list(zip(weather_encoded,temp_encoded))
print(features)

### step 2. sklearn의 Naive Bayes 모델인 MultinomialNB를 사용하여 예측하겠습니다.
- weather = 0(Overcast), temp = 2(Mild)인 경우 play = Yes로 예측됩니다.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# 라플라스 스무딩(alpha = 1)
model = MultinomialNB(alpha=1)

# Train the model using the training sets
model.fit(features,label)

#Predict Output
predicted= model.predict([[0,2]]) # 0:Overcast, 2:Mild
print("Predicted Value:", predicted) # 1: Yes

### 이제 sklearn의 와인 데이터를 바탕으로 MultinomialNB를 활용하여 예측해주세요!
- train과 test 비율은 8:2로 해주세요.
- metrics는 accuracy로 진행해주세요.

In [ ]:
from sklearn import datasets

#Load dataset
wine = datasets.load_wine()

# print the names of the 13 features
print("Features: ", wine.feature_names)

# print the label type of wine(class_0, class_1, class_2)
print("Labels: ", wine.target_names)

In [ ]:
data=pd.concat([pd.DataFrame({'Labels':wine.target}),pd.DataFrame(wine.data, columns = wine.feature_names)], axis=1)
data

In [ ]:
from sklearn.model_selection import train_test_split

X=data.drop(['Labels'],axis=1)
y=data.Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, train_size = 0.2, random_state = 1)   
#random_state-재현가능하도록 난수의 초기값 지정/아무숫자나 가능

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train,y_train)


In [ ]:
from sklearn.metrics import accuracy_score


score=clf.score(X_test,y_test)


print("Accuracy:", score)